In [39]:
import pandas as pd
import os
import sys
from pathlib import Path
from prfndim.prfndim import Z, P, C, R, S
import numpy as np

In [40]:
path = Path.cwd().parent / "data" / "prfndim" / "raw_prfndim.csv"
df = pd.read_csv(path)

arities = []
for index, row in df.iterrows():
    arities.append(eval(row["expr"]).arity)
df["arity"] = arities
print(df)

                                                     expr  arity
0                   C(P(3, 2), P(2, 1), P(2, 2), P(2, 2))    2.0
1                  R(P(2, 1), P(3, 1), P(3, 3), Z(), Z())    1.0
2                   C(P(3, 2), P(4, 3), P(4, 2), P(4, 4))    4.0
3                           C(P(3, 3), P(3, 3), Z(), Z())    3.0
4       R(P(2, 1), P(3, 3), C(P(3, 3), P(3, 3), Z(), Z...    1.0
...                                                   ...    ...
200007                                            P(3, 3)    3.0
200008                                            P(4, 1)    4.0
200009                                            P(4, 2)    4.0
200010                                            P(4, 3)    4.0
200011                                            P(4, 4)    4.0

[200012 rows x 2 columns]


In [41]:
df_unique = df.drop_duplicates(subset=["expr"])
print(df_unique)

                                                     expr  arity
0                   C(P(3, 2), P(2, 1), P(2, 2), P(2, 2))    2.0
1                  R(P(2, 1), P(3, 1), P(3, 3), Z(), Z())    1.0
2                   C(P(3, 2), P(4, 3), P(4, 2), P(4, 4))    4.0
3                           C(P(3, 3), P(3, 3), Z(), Z())    3.0
4       R(P(2, 1), P(3, 3), C(P(3, 3), P(3, 3), Z(), Z...    1.0
...                                                   ...    ...
200007                                            P(3, 3)    3.0
200008                                            P(4, 1)    4.0
200009                                            P(4, 2)    4.0
200010                                            P(4, 3)    4.0
200011                                            P(4, 4)    4.0

[93329 rows x 2 columns]


In [42]:
sample_max = 10
sample_num = 5
max_p_arity = 4

eq_domain = [np.zeros((1))] + [
    np.random.randint(1, sample_max + 1, size=(sample_num, dim))
    for dim in range(1, max_p_arity + 1)
]
eq_domain[1] = np.array(range(1, sample_max + 1)).reshape(-1, 1)

outputs_li = []
inputs_li = []
for index, row in df_unique.iterrows():
    expr = eval(row["expr"])
    if expr.arity is None:
        inputs = np.full((sample_num, 1), 0)
    else:
        inputs = eq_domain[expr.arity]
    outputs = tuple(expr.eval(*args) for args in inputs)
    inputs_li.append(tuple(tuple(int(arg) for arg in args) for args in inputs))
    outputs_li.append(tuple(int(output) for output in outputs))
df_unique["inputs"] = inputs_li
df_unique["outputs"] = outputs_li


df_unique_output = df_unique.drop_duplicates(subset=["arity", "outputs"])



/tmp/ipykernel_7830/321815760.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique["inputs"] = inputs_li
/tmp/ipykernel_7830/321815760.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_unique["outputs"] = outputs_li


In [45]:
df_unique_output = df_unique_output.sort_values(by="arity").reset_index(drop=True)
df_unique_output.to_csv("../data/prfndim/unique_output.csv", index=True)